In [ ]:
##FOUNDATION OF NATURAL LANGUAGE PROCESSING AND MACHINE LEARNING

In [ ]:
##1.Real world problem predict rating given product review on amazaon

In [1]:
%matplotlib inline
import sqlite3 
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve,auc
from nltk.stem.porter import PorterStemmer
con=sqlite3.connect('database.sqlite')
filtered_data=pd.read_sql_query("""
SELECT *
FROM Reviews
WHERE Score !=3
""",con)
def partition(x):
    if x<3:
        return 'negative'
    return 'positive'
actualScore=filtered_data['Score']
positiveNegative=actualScore.map(partition)
filtered_data['Score']=positiveNegative
filtered_data.shape
filtered_data

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,positive,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,negative,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,positive,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,negative,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,positive,1350777600,Great taffy,Great taffy at a great price. There was a wid...
5,6,B006K2ZZ7K,ADT0SRK1MGOEU,Twoapennything,0,0,positive,1342051200,Nice Taffy,I got a wild hair for taffy and ordered this f...
6,7,B006K2ZZ7K,A1SP2KVKFXXRU1,David C. Sullivan,0,0,positive,1340150400,Great! Just as good as the expensive brands!,This saltwater taffy had great flavors and was...
7,8,B006K2ZZ7K,A3JRGQVEQN31IQ,Pamela G. Williams,0,0,positive,1336003200,"Wonderful, tasty taffy",This taffy is so good. It is very soft and ch...
8,9,B000E7L2R4,A1MZYO9TZK0BBI,R. James,1,1,positive,1322006400,Yay Barley,Right now I'm mostly just sprouting this so my...
9,10,B00171APVA,A21BT40VZCCYT4,Carol A. Reed,0,0,positive,1351209600,Healthy Dog Food,This is a very healthy dog food. Good for thei...


In [2]:
## filter data where same user with duplicate Summary','Time','Text' 

filtered_data_duplicates=filtered_data[filtered_data.duplicated(['UserId','ProfileName','Summary','Time','Text'])]

filtered_data_sorted=filtered_data.sort_values(by='ProductId',axis=0,ascending=True)
filtered_data_sorted
filtered_data_duplicates
final_data=filtered_data_sorted.drop_duplicates(subset={'UserId','ProfileName','Summary','Time','Text'}, keep='first', inplace=False)
final_data

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
138706,150524,0006641040,ACITT7DI6IDDL,shari zychinski,0,0,positive,939340800,EVERY book is educational,this witty little book makes my son laugh at l...
138688,150506,0006641040,A2IW4PEEKO2R0U,Tracy,1,1,positive,1194739200,"Love the book, miss the hard cover version","I grew up reading these Sendak books, and watc..."
138689,150507,0006641040,A1S4A3IQ2MU7V4,"sally sue ""sally sue""",1,1,positive,1191456000,chicken soup with rice months,This is a fun way for children to learn their ...
138690,150508,0006641040,AZGXZ2UUK6X,"Catherine Hallberg ""(Kate)""",1,1,positive,1076025600,a good swingy rhythm for reading aloud,This is a great little book to read aloud- it ...
138691,150509,0006641040,A3CMRKGE0P909G,Teresa,3,4,positive,1018396800,A great way to learn the months,This is a book of poetry about the months of t...
138693,150511,0006641040,A1C9K534BCI9GO,Laura Purdie Salas,0,0,positive,1344211200,Charming and childlike,"A charming, rhyming book that describes the ci..."
138694,150512,0006641040,A1DJXZA5V5FFVA,A. Conway,0,0,positive,1338249600,Must have.,I set aside at least an hour each day to read ...
138695,150513,0006641040,ASH0DZQQF6AIZ,tessarat,0,0,positive,1325721600,A classic,I remembered this book from my childhood and g...
138696,150514,0006641040,A2ONB6ZA292PA,Rosalind Matzner,0,0,positive,1313884800,Chicken soup with Rice,It's a great book with adorable illustrations....
138697,150515,0006641040,A2RTT81R6Y3R7X,Lindylu,0,0,positive,1303171200,One of our family's favorite books,This book is a family favorite and was read to...


In [3]:
# checking how much perontage data remains
final_data.size/filtered_data.size * 100

69.4795117665182

### USE COMMON SENSE:

HelpfulnessNumerator <= HelpfulnessDenominator 


In [4]:
#just which are having numarator greter than denomior feed back giving always less that helpful or not.
#get the duplicate fellows
 
final_data_irrelavent=final_data[final_data.HelpfulnessNumerator > final_data.HelpfulnessDenominator]
print(final_data_irrelavent.shape)
final_data_irrelavent

(2, 10)


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
59301,64422,B000MIDROQ,A161DK06JJMCYF,"J. E. Stephens ""Jeanne""",3,1,positive,1224892800,Bought This for My Son at College,My son loves spaghetti so I didn't hesitate or...
41159,44737,B001EQ55RW,A2V0I904FH7ABY,Ram,3,2,positive,1212883200,Pure cocoa taste with crunchy almonds inside,It was almost a 'love at first bite' - the per...


In [5]:
# remove duplicate fellows 
final_data=final_data[final_data.HelpfulnessNumerator <= final_data.HelpfulnessDenominator]
final_data.shape

(365331, 10)

In [7]:
## Positive and negative review
final_data_pos=final_data[final_data['Score']=='positive']
final_data_neg=final_data[final_data['Score']=='negative']
#print(pos.count(axis=1),neg.count(axis=1),final_data.count(axis=1))
#len(df.columns)count
print(final_data_pos.shape[0],final_data_neg.shape[0],final_data_pos.shape[0]+final_data_neg.shape[0])
final_data.shape


307969 57364 365333


(365333, 10)

In [8]:
#### Observations: Here below is to separate  positive / negitive review from final_data, 
       out off 365331 
     - 365331 is positive reviews by the customers
     - 57364  is negative reviews by the customers
     - this step is just to know count for future use.

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 3)

In [ ]:
## Text Preprocessing.

### given 8 feture pridict sentiment  ? wether it positive or Negitive (Critical Review) :
    - Id	
    - ProductId	
    - UserId	
    - ProfileName	
    - HelpfulnessNumerator
    - HelpfulnessDenominator
    - Score	
    - Time	polarity
    - Summary	
    - Text
####  Summary  ,	Text  -- both  a text 
####  convert into vector we can levarage power of algibra .
Big Q how to convert Text into Mathematical Numerical Vectors we can levarage linear lagibra.


In [ ]:
## [4]  - Bag of Words (BoW)

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

count_vector=CountVectorizer()
final_counts=count_vector.fit_transform(final_data['Text'].values)
final_counts
#print(final_counts,final_counts.get_shape(),final_counts.shape,type(final_counts))

<365333x115282 sparse matrix of type '<class 'numpy.int64'>'
	with 19420875 stored elements in Compressed Sparse Row format>

In [ ]:
##[5] Text Preprocessing (Code Sample)

Now that we have finished deduplication our data requires some preprocessing before we go on further with analysis and making the prediction model.

Hence in the Preprocessing phase we do the following in the order below:-

1. Begin by removing the html tags
2. Remove any punctuations or limited set of special characters like , or . or # etc.
3. Check if the word is made up of english letters and is not alpha-numeric
4. Check to see if the length of the word is greater than 2 (as it was researched that there is no adjective in 2-letters)
5. Convert the word to lowercase
6. Remove Stopwords
7. Finally Snowball Stemming the word (it was obsereved to be better than Porter Stemming)<br>

After which we collect the words used to describe positive and negative reviews

In [9]:
import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
stop=set(stopwords.words('english'))
sno=nltk.stem.SnowballStemmer('english')
def cleanhtml(sentence):
    cleanr=re.compile('<.*?>')
    cleantext=re.sub(cleanr, ' ',sentence)
    return cleantext
def cleanpunc(sentence):
    cleaned=re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned=re.sub(r'[.|,|)|(|\|/)]',r' ',cleaned)
    return cleaned
print(stop)
                   

{'while', 'its', 'being', 'my', 'on', "don't", 'out', "haven't", 'or', 'mightn', 'yours', 'should', 've', 'was', "hadn't", 'as', 'if', 'to', 'is', 'some', "shouldn't", 'into', "isn't", 'in', 'm', "wasn't", 'wasn', 'don', 'down', 'he', 'yourselves', 'after', 'so', 'wouldn', 'hers', "couldn't", 'that', 're', 'before', 'we', 'and', 'other', 'can', 'are', "shan't", 'yourself', "doesn't", 'just', 'won', "won't", 'nor', 'their', 'am', 'own', "you've", 't', 'herself', "mustn't", 'me', 'him', 'i', 'between', 'his', 'above', 's', 'when', 'ourselves', 'from', "you'll", 'has', 'through', 'against', 'off', 'did', 'they', 'for', 'all', 'because', 'whom', 'ma', 'now', 'be', 'needn', "weren't", 'doesn', 'you', 'with', 'themselves', 'only', 'during', 'more', 'myself', 'what', 'how', 'do', 'these', 'no', 'why', 'our', 'weren', 'her', 'hadn', 'an', 'ain', 'each', "didn't", 'any', 'she', 'where', 'himself', 'than', 'y', "hasn't", 'itself', 'have', 'having', 'mustn', 'most', 'it', 'up', 'which', 'been', '

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

count_vector=CountVectorizer()
final_counts=count_vector.fit_transform(filtered_data['Text'])
final_counts
print(final_counts,final_counts.get_shape(),final_counts.shape,type(final_counts))

  (0, 70828)	1
  (0, 103750)	1
  (0, 9254)	1
  (0, 93405)	1
  (0, 45666)	1
  (0, 59142)	1
  (0, 62513)	1
  (0, 71725)	1
  (0, 20421)	2
  (0, 95555)	1
  (0, 59284)	1
  (0, 67922)	1
  (0, 83118)	1
  (0, 103305)	2
  (0, 98734)	1
  (0, 64055)	1
  (0, 70655)	1
  (0, 64968)	1
  (0, 83208)	2
  (0, 84765)	1
  (0, 50620)	1
  (0, 19419)	1
  (0, 104543)	1
  (0, 7296)	1
  (0, 103444)	1
  :	:
  (525812, 57052)	1
  (525812, 50620)	1
  (525812, 19419)	1
  (525812, 104543)	1
  (525812, 7296)	1
  (525812, 8302)	2
  (525812, 38632)	1
  (525812, 103374)	2
  (525813, 49265)	1
  (525813, 110239)	1
  (525813, 26631)	1
  (525813, 101251)	1
  (525813, 85724)	1
  (525813, 6211)	1
  (525813, 91121)	1
  (525813, 7750)	1
  (525813, 108904)	1
  (525813, 75332)	1
  (525813, 109947)	1
  (525813, 112938)	1
  (525813, 9973)	2
  (525813, 59142)	1
  (525813, 59284)	1
  (525813, 83208)	1
  (525813, 8302)	1 (525814, 115282) (525814, 115282) <class 'scipy.sparse.csr.csr_matrix'>


In [10]:
import nltk
# open termianal python run python terminal 
## python
## import nltk nltk.download()
## nltk.download('stopwords')
##https://www.geeksforgeeks.org/regular-expression-python-examples-set-1/
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 
from nltk.stem import WordNetLemmatizer
import string
### clean punctuations Clean Html


### set of stopWOrds  snowball Stemmer 

stop_word=set(stopwords.words('english'))
snow_stemmer=nltk.stem.SnowballStemmer("english")

### clean html clean spaces
def clean_html(sentence_in) :
    clear_patteren=re.compile(r'<.*?>') # pattern matcher
    clean_text=clear_patteren.sub(' ',sentence_in)
    return clean_text
    
def clean_punc(word_in) :
    cleaned =re.sub(r'[?|!|\'|#|]',r'',word_in) # pattern matcher
    cleaned =re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)  ## . , \ / means  a composite or two word so splitting with ' '
    return cleaned
print(stop_word)
print('**************')
print(snow_stemmer.stem('tasty'))
s="sentence this witty little book makes <html> </html>my son laugh at loud. i recite it in the car as we're driving along and he always can sing the refrain. he's learned about whales, India, drooping roses: "
print('clean_html    :',clean_html(s) ,'\n')
print('clean_punc    :',clean_punc(clean_html(s)))

#print(final_data['Text'].values[0:5])
print(final_data.shape[0])
print(len(final_data['Text']))
#final_data['Text'][1:1000].values

{'while', 'its', 'being', 'my', 'on', "don't", 'out', "haven't", 'or', 'mightn', 'yours', 'should', 've', 'was', "hadn't", 'as', 'if', 'to', 'is', 'some', "shouldn't", 'into', "isn't", 'in', 'm', "wasn't", 'wasn', 'don', 'down', 'he', 'yourselves', 'after', 'so', 'wouldn', 'hers', "couldn't", 'that', 're', 'before', 'we', 'and', 'other', 'can', 'are', "shan't", 'yourself', "doesn't", 'just', 'won', "won't", 'nor', 'their', 'am', 'own', "you've", 't', 'herself', "mustn't", 'me', 'him', 'i', 'between', 'his', 'above', 's', 'when', 'ourselves', 'from', "you'll", 'has', 'through', 'against', 'off', 'did', 'they', 'for', 'all', 'because', 'whom', 'ma', 'now', 'be', 'needn', "weren't", 'doesn', 'you', 'with', 'themselves', 'only', 'during', 'more', 'myself', 'what', 'how', 'do', 'these', 'no', 'why', 'our', 'weren', 'her', 'hadn', 'an', 'ain', 'each', "didn't", 'any', 'she', 'where', 'himself', 'than', 'y', "hasn't", 'itself', 'have', 'having', 'mustn', 'most', 'it', 'up', 'which', 'been', '

In [12]:
import re
sentences=filtered_data['Text'].values
# in this list of sentences find html tags print word contain HTML  import regex from re  '<.*?>' is html tag
sentence_contain_html=0

for sentence in sentences:
    if(re.findall('<.*?>',sentence)):
        sentence_contain_html=sentence_contain_html+1
        print(sentence+'\n')
        break;
sentence_contain_html
    

I don't know if it's the cactus or the tequila or just the unique combination of ingredients, but the flavour of this hot sauce makes it one of a kind!  We picked up a bottle once on a trip we were on and brought it back home with us and were totally blown away!  When we realized that we simply couldn't find it anywhere in our city we were bummed.<br /><br />Now, because of the magic of the internet, we have a case of the sauce and are ecstatic because of it.<br /><br />If you love hot sauce..I mean really love hot sauce, but don't want a sauce that tastelessly burns your throat, grab a bottle of Tequila Picante Gourmet de Inclan.  Just realize that once you taste it, you will never want to use any other sauce.<br /><br />Thank you for the personal, incredible service!



1

In [19]:
i=0
s=''
final_string=[]
all_postive_words=[]
all_negitive_words=[]
for sentence in final_data['Text'].values :
    filterted_sentences=[]
    sentence=clean_html(sentence)
    for wordz in sentence.split():
        for cleaned_word in clean_punc(wordz).split():
            if (cleaned_word.isalpha() & ( len(cleaned_word) > 2) ) :
                if (cleaned_word not in stop_word):
                    s=(snow_stemmer.stem(cleaned_word.lower())).encode('utf-8')
                    filterted_sentences.append(s)
                    if ( final_data['Score'].values[i] == 'positive' ) :
                        all_postive_words.append(s)
                    if ( final_data['Score'].values[i] == 'negative' ) :
                        all_negitive_words.append(s)
                else:
                    continue
            else:
                continue
              
    i=i+1
    review_str=b"".join(filterted_sentences)
    final_string.append(review_str)
print(len(final_string))
final_data['CleanedText']=final_string 
final_data.head(6)
print('done')

365333
done


C:\Users\jasmin shaik\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [14]:
freq_dist_all_postive_words=nltk.FreqDist(all_postive_words)
freq_dist_all_negative_words=nltk.FreqDist(all_negitive_words)
print('Most Common Positive Words : ',freq_dist_all_postive_words.most_common(20))
print('Most Common Negative Words : ',freq_dist_all_negative_words.most_common(20))

NameError: name 'all_postive_words' is not defined

In [20]:
#bi-gram,tri-gram and n-gram
#removing stop word like "not" should be avoided building n-grams
#It is possible that you have an older version of sklearn installed. 

#count_vector=CountVectorizer(ngram_range=(1,2))
count_vector=CountVectorizer(vocabulary=vocabulary, ngram_range=(1, 2))
final_bigram_counts=count_vector.fit_transform(filtered_data['Text'].values)
final_bigram_counts

NameError: name 'vocabulary' is not defined

In [21]:
tf_idf_vect=TfidfVectorizer(ngram_range=(1,2))
final_tf_idf=tf_idf_vect.fit_transform(filtered_data['Text'].values)

In [22]:
features=tf_idf_vect.get_feature_names()
len(features)
print(features[100:120])

['00 cans', '00 cappuccino', '00 capsules', '00 carton', '00 case', '00 cd', '00 chai', '00 chance', '00 change', '00 cheaper', '00 check', '00 chocolate', '00 clip', '00 coffee', '00 coles', '00 compared', '00 complete', '00 considering', '00 container', '00 cookie']


In [23]:
print(features[10000:10050])

['14 count', '14 country', '14 crackers', '14 credit', '14 crude', '14 cup', '14 cupcakes', '14 cups', '14 currants', '14 dad', '14 daily', '14 day', '14 days', '14 decaf', '14 delicious', '14 didn', '14 dietary', '14 different', '14 disks', '14 dogs', '14 dollars', '14 don', '14 down', '14 dozen', '14 drops', '14 due', '14 dv', '14 each', '14 eating', '14 enjoy', '14 except', '14 fat', '14 few', '14 fiber', '14 fighting', '14 fl', '14 flavors', '14 flax', '14 flies', '14 floor', '14 fluid', '14 for', '14 fourteen', '14 from', '14 fruit', '14 gallons', '14 garden', '14 gauge', '14 give', '14 going']


In [26]:
#https://buhrmann.github.io/tfidf-analysis.html
def top_tfidf_feats(row, features, top_n=25):
    ''' Get top n tfidf values in row and return them with their corresponding feature names.'''
    topn_ids = np.argsort(row)[::-1][:top_n]
    top_feats = [(features[i], row[i]) for i in topn_ids]
    df = pd.DataFrame(top_feats)
    df.columns = ['feature', 'tfidf']
    return df


In [27]:
#final_tf_idf reviews and vctor matrix 
top_tfidf=top_tfidf_feats(final_tf_idf[1,:].toarray()[0],features,25)
top_tfidf

,feature,tfidf
0,as jumbo,0.395126
1,jumbo,0.263072
2,sized unsalted,0.203690
3,unsalted not,0.203690
4,jumbo salted,0.203690
5,vendor intended,0.203690
6,arrived labeled,0.189844
7,peanuts,0.189165
8,actually small,0.187089
9,error or,0.169671


# [8]  17.15 - Word2Vec (Code Sample

In [ ]:
###### Here we are using pretarined model by google 
https://code.google.com/archive/p/word2vec/

Google’s pre-trained Word2Vec model up and running in Python to play with.
its 3.3 gb once you load into your memory it occupyies ~9GB.so please do this step if you have >12G of ram

As an interface to word2vec, I decided to go with a Python package called gensim. gensim appears to be a popular NLP package, and has some nice documentation and tutorials, including for word2vec.

You can download Google’s pre-trained model here. It’s 1.5GB! It includes word vectors for a vocabulary of 3 million words and phrases that they trained on roughly 100 billion words from a Google News dataset. The vector length is 300 features.

Loading this model using gensim is a piece of cake; you just need to pass in the path to the model file (update the path in the code below to wherever you’ve placed the file).

https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit

In [28]:
### run below in CMD / Terminal /github desktop
##### process of installation of gensim in anaconda
1.enter contorl+c
2.enter pwd
3.enter ls
4.enter gensim
5.enter pip(wait for time)
6.enter as ===>conda install -c conda-forge gensim(it takes time to install)
7.proceed([y]/n)? then enter y
8.wait for some time it will display done

pip install -U gensim

SyntaxError: invalid syntax (<ipython-input-28-6cf67d324e13>, line 3)

In [25]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle

model=KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin',binary=True)


PermissionError: [Errno 13] Permission denied: 'GoogleNews-vectors-negative300.bin'

In [ ]:
#model vector for computer
print(model.wv['Random'][5])
print(model.wv['Rabbani'][5])
#model vector for hello
print(model.wv.similarity('woman','man'))
print(model.wv.most_similar('woman'))
print('Rabbani : ',model.wv.most_similar('Rabbani'))
#"word 'tasti' not in vocabulary"
#print('tasti : ',model.wv.most_similar('tasti')) 
#similarity large words are more closer
print('tasti : ',model.wv.most_similar('tasty'))


In [ ]:
print(final_data['Text'].values[0])
print('*********************')

### now we are going to use our own model on our Reviews system

In [ ]:
# train your own word to vec model using your own text corpus.
# using our own text corpus
#w2v there might possiblity of not having same stemmed words
import gensim 

i=0
list_of_sent=[]
for sent in final_data['Text'].values:
    filterted_sentences=[]
    sent =clean_html(sent)
    for wordz in sent.split(): 
        for cleaned_word in clean_punc(wordz).split():
            if (cleaned_word.isalpha() ) :
                filterted_sentences.append(cleaned_word.lower())
            else :
                continue;
    list_of_sent.append(filterted_sentences)    
list_of_sent[0][1:5]

In [ ]:
print(final['Text'].values[0])
print("*****************************************")
print(list_of_sent[0])

In [ ]:
#at lest 5 time then only convert 
#multicore  box workers 4
#train to word vec model mincount count mode than this then only convert . 
#ttakes more time
import gensim
w2v_model=gensim.models.Word2Vec(list_of_sent,min_count=5,size=50,workers=4)
w2v_model

In [ ]:
words=list(w2v_model.wv.vocab)
print(len(words))

In [ ]:
#Here we are using our own data set i.e 364k data set 
w2v_model.wv.most_similar('tasty')

In [ ]:
w2v_model.wv.most_similar('like')

# [9] 17.16 - Avg-Word2Vec and TFIDF-Word2Vec (Code Sample)

In [29]:
# AVG -word 2 vec 
# compute avg word 2 vec for each review 
# dont worry below snippet will take time :) 
list_sentence_vector=[] # the Avg-Word2Vec in each sentence/review is stord in this . 
for sentence in  list_of_sent:     # for each review / sentence
    sentence_vector=np.zeros(100)   # as word vector are of zero length
    cnt_words=0                     #num of words with valid vector in the sentence review 
    for word in sentence:               # for each word in for each review / sentence
        try:
            w2v_vec=w2v_model.wv[word]
            # sentence_vector += w2v_vec
            sentence_vector = sentence_vector+w2v_vec
            # cnt_words += 1
            cnt_words =cnt_words+1
        except:
            pass
    #sentence_vector/=cnt_words
    sentence_vector=sentence_vector/cnt_words;  # avaergae 
    list_sentence_vector.append(sentence_vector)
print('---')
print(len(list_sentence_vector))
print(len(list_sentence_vector[0]))

NameError: name 'list_of_sent' is not defined

In [ ]:
#tfidf_feat weighted word 
tf_idf_feat=tf_idf_vector.get_feature_names() #tfidf words/columns
# AVG -word 2 vec 
# compute avg word 2 vec for each review 
list_sentence_vectors=[] # the Avg-Word2Vec in each sentence/review is stord in this . 
row=0
for sentence in  list_of_sent:     # for each review / sentence
    sentence_vector=np.zeros(50)   # as word vector are of zero length
    cnt_words=0
    for word in sent:
        try:
            w2v_vec=w2v_model.wv[word] 
            #Obtain the tf_idf word in sentence/ review
            tf_idf=final_tf_idf[row,tf_idf_feat.index(word)]
            sentence_vector = sentence_vector+w2v_vec*tf_idf
            cnt_words =cnt_words+1
        except:
            pass
    sentence_vector=sentence_vector/cnt_words
    list_sentence_vector.append(sentence_vector)
    row += 1
print(len(list_sentence_vector))
print(len(list_sentence_vector[0]))